# MetaCheX

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tensorflow as tf
import tensorflow_addons as tfa
from glob import glob
import pickle
from sklearn.metrics import roc_curve
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

# metachex materials
from metachex.configs.config import *
from metachex.dataloader import MetaChexDataset
from metachex.loss import Losses

In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"]=""
tf.test.is_gpu_available()
physical_devices = tf.config.experimental.list_physical_devices('GPU')
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2021-11-13 19:29:56.326590: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-13 19:29:56.378905: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-13 19:29:56.388035: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-13 19:29:56.388783: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

### Instantiate Dataset and Splits

In [3]:
# Instantiate dataset
dataset = MetaChexDataset()
# Get class weights (feed this into model construction for weighted loss)
# indiv_class_weights, combo_class_weights = dataset.get_class_weights()
# Grab training dataset
train_ds = dataset.train_ds

[INFO] pre-processing
Data already processed. Loading from save data/data.pkl
[INFO] truncating dataset
[INFO] constructing tf train/val/test vars
[INFO] shuffle & batch
train/val overlap:  15
train/test overlap;  4
val/test overlap;  12
[    0 10433  3834  2373     0  4048     0  1959 12005  2200  1437     0
   160     0 18064     6    17    10  4809    16     0  5061    15    12
  5666  2902    51  3462  4760    24     0    33    14     9]
classes not trained on: (array([ 0,  4,  6, 11, 13, 20, 30]),)
[   0  629  414  141    0  278    0  123  715  109   80    0   15    0
 1142    0    0    0  601    1    0  339    0    1  335  170    0  278
  258    2    0    2    2    0]
classes not validated on: (array([ 0,  4,  6, 11, 13, 15, 16, 17, 20, 22, 26, 30, 33]),)
[  0 139 426  55   0  72   0  26 195  21  25   0   2   0 304   1   1   4
 602   1   0  70   3   1  86  47   3 245  44   2   0   3   2   1]
classes not tested on: (array([ 0,  4,  6, 11, 13, 20, 30]),)
self.y.shape:  (113500, 34)

In [4]:
# elaine_repo_path = '/home/elainesui/MetaCheX'
# os.chdir(elaine_repo_path)

In [5]:
images, labels = next(iter(train_ds))
images.shape

(8, 224, 224, 3)

### Get data stats: 
- Number of images with each label (individual)
- Number of unique labels (individual)
- Number of labels total (including combos)

In [6]:
# Check out the data
unique_labels_dict, df_combo_counts, df_label_nums, df_combo_nums = dataset.get_data_stats(dataset.df)

print("Number of total images: ", df_combo_nums['count'].sum())
print("Number of total individual labels (includes 'No Finding'): ", df_label_nums.shape[0])
print("Number of total label combos (includes individual labels): ", df_combo_nums.shape[0])
print("****************************")
print("Number of images with each individual label")
display(df_label_nums)
print("\n")
print("Number of images with each combo label (Bottom 20)")
display(df_combo_nums.tail(20))
print("\n")
display(df_combo_counts.head())

Number of total images:  134151
Number of total individual labels (includes 'No Finding'):  35
Number of total label combos (includes individual labels):  821
****************************
Number of images with each individual label


,count
No Finding,70575
Infiltration,19894
Effusion,13317
Atelectasis,11559
Nodule,6331
Lung_Opacity,6012
Mass,5782
Pneumothorax,5302
Consolidation,4667
COVID-19,4200




Number of images with each combo label (Bottom 20)


,count
Consolidation|Effusion|Infiltration|Mass|Pneumothorax,1
Atelectasis|Consolidation|Effusion|Nodule|Pneumothorax,1
Effusion|Emphysema|Infiltration|Mass,1
Atelectasis|Consolidation|Effusion|Emphysema|Mass|Pneumothorax,1
Cardiomegaly|Effusion|Pleural_Thickening|Pneumothorax,1
Atelectasis|Effusion|Fibrosis|Infiltration|Nodule,1
Effusion|Fibrosis|Mass|Pleural_Thickening,1
Cardiomegaly|Consolidation|Effusion|Infiltration|Mass|Pleural_Thickening,1
Atelectasis|Mass|Nodule|Pneumonia,1
Atelectasis|Emphysema|Fibrosis|Infiltration,1


,count interval,number of labels
0,< 5,493
1,"[5, 100)",266
2,"[100, 1k)",46
3,"[1k, 10k)",15
4,>= 10k,1


### Get updated data stats

In [7]:
print("Stats for condensed dataset")
print("---------------------------")
unique_labels_dict, df_combo_counts, df_label_nums, df_combo_nums = dataset.get_data_stats(dataset.df_condensed)
print(dataset.df_condensed['label_multitask'][0].shape)

print("Number of total images: ", df_combo_nums['count'].sum())
print("Number of total individual labels (includes 'No Finding'): ", df_label_nums.shape[0])
print("Number of total label combos (includes individual labels): ", df_combo_nums.shape[0])
print("****************************")
print("Number of images with each individual label")
display(df_label_nums)
print("\n")
print("Number of images with each combo label (Bottom 20)")
display(df_combo_nums.tail(20))
print("\n")
display(df_combo_counts.head())

## Just making sure that the weights match with the correct labels
indiv = df_label_nums
indiv_weights = (1 / indiv['count']) * (indiv['count'].sum() / indiv.shape[0])
indiv_weights = indiv_weights.sort_index()
indiv_weights = indiv_weights.drop(['No Finding'])
display(indiv_weights)
indiv_class_weights = dict(list(enumerate(indiv_weights.values)))
display(indiv_class_weights)

unique_labels = list(unique_labels_dict.keys())
unique_labels.remove('No Finding')
unique_labels.sort() ## alphabetical order
print(unique_labels)

Stats for condensed dataset
---------------------------
(34,)
Number of total images:  133318
Number of total individual labels (includes 'No Finding'):  28
Number of total label combos (includes individual labels):  328
****************************
Number of images with each individual label


,count
No Finding,70575
Infiltration,19510
Effusion,12915
Atelectasis,11201
Nodule,6087
Lung_Opacity,6012
Mass,5470
Pneumothorax,5062
Consolidation,4398
COVID-19,4200




Number of images with each combo label (Bottom 20)


,count
Cardiomegaly|Effusion|Emphysema|Pneumothorax,5
Pneumonia|Pneumothorax,5
Atelectasis|Consolidation|Effusion|Emphysema,5
Atelectasis|Consolidation|Effusion|Infiltration|Mass,5
Cardiomegaly|Effusion|Infiltration|Pleural_Thickening,5
Effusion|Emphysema|Nodule,5
Emphysema|Mass|Nodule,5
Effusion|Mass|Pleural_Thickening|Pneumothorax,5
Atelectasis|Effusion|Mass|Pleural_Thickening,5
Emphysema|Pneumonia,5


,count interval,number of labels
0,< 5,0
1,"[5, 100)",266
2,"[100, 1k)",46
3,"[1k, 10k)",15
4,>= 10k,1


Atelectasis              0.512840
COVID-19                 1.367696
Cardiomegaly             2.236015
Consolidation            1.306121
Edema                    2.725010
Effusion                 0.444779
Emphysema                2.465374
Fibrosis                 3.725241
Hernia                  32.453793
Infiltration             0.294430
Influenza             1148.864286
Klebsiella             574.432143
Legionella             574.432143
Lung_Opacity             0.955476
MERS-CoV               574.432143
Mass                     1.050150
Mycoplasma             522.211039
Nocardia               718.040179
Nodule                   0.943703
Pleural_Thickening       1.841719
Pneumocystis           191.477381
Pneumonia                1.679626
Pneumothorax             1.134793
SARS                   359.020089
Streptococcus          261.105519
Tuberculosis           319.128968
Varicella              957.386905
Name: count, dtype: float64

{0: 0.5128400525463288,
 1: 1.3676955782312925,
 2: 2.2360145693154645,
 3: 1.3061212888975509,
 4: 2.725010165356465,
 5: 0.4447790498313146,
 6: 2.4653740036787246,
 7: 3.7252408745599404,
 8: 32.45379338175948,
 9: 0.2944295965438969,
 10: 1148.8642857142856,
 11: 574.4321428571428,
 12: 574.4321428571428,
 13: 0.9554759528561924,
 14: 574.4321428571428,
 15: 1.0501501697571167,
 16: 522.211038961039,
 17: 718.0401785714286,
 18: 0.9437032082423901,
 19: 1.8417189575413364,
 20: 191.47738095238094,
 21: 1.679626148705096,
 22: 1.1347928543207089,
 23: 359.0200892857143,
 24: 261.1055194805195,
 25: 319.12896825396825,
 26: 957.3869047619047}

['Atelectasis', 'COVID-19', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Influenza', 'Klebsiella', 'Legionella', 'Lung_Opacity', 'MERS-CoV', 'Mass', 'Mycoplasma', 'Nocardia', 'Nodule', 'Pleural_Thickening', 'Pneumocystis', 'Pneumonia', 'Pneumothorax', 'SARS', 'Streptococcus', 'Tuberculosis', 'Varicella']


## Step 2: Finetuned CheXNet Baseline

Note: CheXNet = DenseNet121 trained on ChestX-ray14 dataset (multi-task binary classification)

Pre-trained weights: https://github.com/brucechou1983/CheXNet-Keras

In [8]:
def load_chexnet_pretrained(class_names=np.arange(14), weights_path='chexnet_weights.h5', 
                            input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)):

    img_input = tf.keras.layers.Input(shape=input_shape)
    base_model = tf.keras.applications.densenet.DenseNet121(include_top=False, weights=None, 
                                                            input_tensor=img_input, pooling='avg')
    base_model.trainable = True


    x = base_model.output
    predictions = tf.keras.layers.Dense(len(class_names), activation="sigmoid", name="predictions")(x)
    model = tf.keras.models.Model(inputs=img_input, outputs=predictions)
    model.load_weights(weights_path)

    return model


def load_chexnet(output_dim):
    """
    output_dim: dimension of output
    """
    
    base_model_old = load_chexnet_pretrained()
    x = base_model_old.layers[-2].output ## remove old prediction layer
    
    ## The prediction head can be more complicated if you want
    predictions = tf.keras.layers.Dense(output_dim, name='prediction', activation='sigmoid')(x)
    chexnet = tf.keras.models.Model(inputs=base_model_old.inputs,outputs=predictions)
    return chexnet
    
chexnet = load_chexnet(dataset.num_classes_multitask)
print(chexnet.summary())

2021-11-13 19:30:00.058406: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-13 19:30:00.058928: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-13 19:30:00.059302: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-13 19:30:00.059702: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-13 19:30:00.060081: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [9]:
chexnet.layers[-1].name

'prediction'

In [10]:
train_ds

### Train baseline -- multi-task binary classification

In [11]:
checkpoint_path = "training_progress/cp.ckpt" # path for saving model weights
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [12]:
class_weights, indiv_class_weights, _ = dataset.get_class_weights(one_cap=True)

loss_fn = Losses(class_weights, batch_size=dataset.batch_size)

# unique_labels = list(unique_labels_dict.keys())
# unique_labels.remove('No Finding')
# unique_labels.sort() ## alphabetical order

# output_dim = len(unique_labels)
chexnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                  loss=loss_fn.weighted_binary_crossentropy(),
#                   loss_weights=1e5,
#                 loss='binary_crossentropy',
                metrics=[tf.keras.metrics.AUC(multi_label=True),  'binary_accuracy', 'accuracy', 
                         tfa.metrics.F1Score(average='micro',num_classes=dataset.num_classes_multitask), 
                         tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                run_eagerly=True)

epochs = 150
hist = chexnet.fit(dataset.train_ds,
            validation_data=dataset.val_ds,
            epochs=epochs,
            steps_per_epoch=dataset.train_steps_per_epoch, ## size(train_ds) * 0.125 * 0.1
            validation_steps=dataset.val_steps_per_epoch, ## size(val_ds) * 0.125 * 0.2
            batch_size=dataset.batch_size ## 8
# #             class_weight=class_weights,
#             callbacks=[cp_callback]
                  )

# with open('./trainHistoryDict', 'wb') as file_pi:
#         pickle.dump(hist.history, file_pi)

2021-11-13 19:30:02.587031: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/150


2021-11-13 19:30:03.779530: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204
2021-11-13 19:30:04.637424: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-11-13 19:30:05.441611: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2021-11-13 19:30:05.662283: W tensorflow/core/framework/op_kernel.cc:1680] Invalid argument: required broadcastable shapes


InvalidArgumentError: in user code:

    /home/edwin/research/code/MetaCheX/metachex/loss.py:38 weighted_loss  *
        return new_weight*bce(y_true_fat, y_pred_fat)
    /home/edwin/anaconda3/envs/metachex/lib/python3.8/site-packages/keras/losses.py:141 __call__  **
        losses = call_fn(y_true, y_pred)
    /home/edwin/anaconda3/envs/metachex/lib/python3.8/site-packages/keras/losses.py:245 call
        return ag_fn(y_true, y_pred, **self._fn_kwargs)
    /home/edwin/anaconda3/envs/metachex/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:206 wrapper
        return target(*args, **kwargs)
    /home/edwin/anaconda3/envs/metachex/lib/python3.8/site-packages/keras/losses.py:1809 binary_crossentropy
        backend.binary_crossentropy(y_true, y_pred, from_logits=from_logits),
    /home/edwin/anaconda3/envs/metachex/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:206 wrapper
        return target(*args, **kwargs)
    /home/edwin/anaconda3/envs/metachex/lib/pyt

In [ ]:
m = tf.keras.metrics.BinaryAccuracy()
m.update_state([[1, 0], [1, 0]], [[1,1], [0, 0]])
m.result().numpy()

In [ ]:
class_weights, _, _ = dataset.get_class_weights()
chexnet = load_chexnet(dataset.num_classes_multitask)
loss_fn = Losses(class_weights)

unique_labels = list(unique_labels_dict.keys())
unique_labels.remove('No Finding')
unique_labels.sort() ## alphabetical order

output_dim = len(unique_labels)
chexnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                loss=loss_fn.weighted_binary_crossentropy(),
                #loss='binary_crossentropy',
                metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

### Evaluate CE baseline

In [ ]:
chexnet.load_weights(checkpoint_path)
print(chexnet.evaluate(dataset.test_ds))

## Precision-Recall Plots (per class)

In [ ]:
df.label_nums.keys()

In [ ]:
class_name = df.label_nums.keys()
dimension = list(range(0,28))
dim_name_mapping = dict(zip(dimension,class_name))

In [ ]:
def precision_recall_curve(y_test, y_pred, class_dim, threshold):
    # Calculate precision - recall over varying threshold
    class_labels_test = y_test[:, class_dim] # list of assuming row vectors!
    class_labels_pred = np.copy(y_pred[:, class_dim])   # list of assuming row vectors!
    class_labels_pred[class_labels_pred >= threshold] = 1 # cast to 1 or 0
    class_labels_pred[class_labels_pred < threshold] = 0 # cast to 1 or 0
    precision, recall, thresholds = precision_recall_curve(class_labels_test, class_labels_pred)
    
    title = 'Precision vs. Recall for' + str(unique_labels[class_dim])# TODO: index into disease name
    plt.plot(recall, precision, color='turquoise', title=title)

## Precision-Recall Stats

In [ ]:
def precision_recall_stats(y_test, y_pred, class_dim, threshold):
    # Calculate current precision recall stat
    pass 

## Visualization

In [ ]:
# tsne visualization of test img features?